In [1]:
# Example usage (text generation):
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

# Specify the model name from Hugging Face Model Hub
model_name = "gpt2"  # Replace with your model name

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)


/opt/anaconda3/envs/capstone/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# Set pad token if it's not already set
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    model.resize_token_embeddings(len(tokenizer))

# Select device based on availability
device = torch.device("mps") if torch.backends.mps.is_available() else torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

# Move model to the selected device
model = model.to(device)

# Example prompt
prompt = "Write a short story about a cat"

# Tokenize the input prompt
encoded_input = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True)

# Move input tensors to the selected device
input_ids = encoded_input['input_ids'].to(device)
attention_mask = encoded_input['attention_mask'].to(device)

# Generate text with attention mask
output = model.generate(
    input_ids=input_ids,
    attention_mask=attention_mask,
    max_length=150,
    num_return_sequences=1,
    pad_token_id=tokenizer.eos_token_id  # Ensure pad_token_id is set
)

# Decode the generated text
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
print(generated_text)

Write a short story about a cat that you love.

A cat is a cat that loves you.

A cat is a cat that loves you.

A cat is a cat that loves you.

A cat is a cat that loves you.

A cat is a cat that loves you.

A cat is a cat that loves you.

A cat is a cat that loves you.

A cat is a cat that loves you.

A cat is a cat that loves you.

A cat is a cat that loves you.

A cat is a cat that loves you.

A cat is a cat that loves you.

A cat is a cat


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, Trainer, TrainingArguments
from datasets import load_dataset
import torch

# Configuration - Adapt these to your specific needs
MODEL_NAME = "google/flan-t5-base"  # Or another suitable pre-trained model from Hugging Face
TRAINING_DATASET = "path/to/your/train.txt"  # Local path or Hugging Face dataset name
VALIDATION_DATASET = "path/to/your/validation.txt" # Add validation dataset
OUTPUT_DIR = "./results"
TUNED_MODEL_NAME = "flan-t5-base-fine-tuned-v1"  # How you'll refer to your fine-tuned model

# Load the dataset
dataset = load_dataset("text", data_files={"train": TRAINING_DATASET, "validation": VALIDATION_DATASET})

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME)

# Set pad token if it's not already set
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    model.resize_token_embeddings(len(tokenizer))

# Select device based on availability
device = torch.device("mps") if torch.backends.mps.is_available() else torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

# Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=128)



In [ ]:
tokenized_datasets = dataset.map(tokenize_function, batched=True)

# Define training arguments
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    num_train_epochs=3,  # Adjust as needed
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    save_steps=1000,
    save_total_limit=2,
    evaluation_strategy="steps", # Add evaluation strategy
)

# Create Trainer instance
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
)

# Train the model
trainer.train()

# Save the fine-tuned model
model.save_pretrained(TUNED_MODEL_NAME)
tokenizer.save_pretrained(TUNED_MODEL_NAME)

# Load the fine-tuned model for inference
tuned_model = AutoModelForCausalLM.from_pretrained(TUNED_MODEL_NAME)
tuned_tokenizer = AutoTokenizer.from_pretrained(TUNED_MODEL_NAME)

# Example usage
prompt = "What is a LLM?"
input_ids = tuned_tokenizer.encode(prompt, return_tensors="pt")

# Check if CUDA is available and move the model and input tensors to the appropriate device
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
tuned_model = tuned_model.to(device)
input_ids = input_ids.to(device)

output = tuned_model.generate(input_ids, max_length=50, num_return_sequences=1)
generated_text = tuned_tokenizer.decode(output[0], skip_special_tokens=True)
print(generated_text)